In [79]:
import importlib
import os
from pathlib import Path

import pandas as pd

import config
import module

try:
    importlib.reload(config)  # reload module
except NameError:
    pass
try:
    importlib.reload(module)  # reload module
except NameError:
    pass

In [80]:
# load df
folder_path = Path("input", config.month)

excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xls')]

dfs = []

for file in excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, skiprows=6)
    dfs.append(df)

df_ori = pd.concat(dfs, ignore_index=True)

WARNING *** file size (2679395) not 512 + multiple of sector size (512)
WARNING *** file size (680547) not 512 + multiple of sector size (512)
WARNING *** file size (2572899) not 512 + multiple of sector size (512)
WARNING *** file size (2679395) not 512 + multiple of sector size (512)
WARNING *** file size (705123) not 512 + multiple of sector size (512)
WARNING *** file size (2572899) not 512 + multiple of sector size (512)
WARNING *** file size (680547) not 512 + multiple of sector size (512)
WARNING *** file size (1491555) not 512 + multiple of sector size (512)
WARNING *** file size (705123) not 512 + multiple of sector size (512)
WARNING *** file size (2695779) not 512 + multiple of sector size (512)


# Clean DF

In [81]:
df_clean = (df_ori
    .dropna(how="all", axis="columns")
    .dropna(how="all", axis="rows")
    .rename(columns=lambda c: c.lower().replace(" ", "_"))  # replace space with _
    .assign(
        student_code=lambda df_: (
            df_["last_name"].str.upper()
            + " "
            + df_["first_name"].str.upper()
            + " - "
            + df_["student_code"].astype("str")
        ).str.strip(),
        student_membership=lambda df_: module.create_student_membership(df_),
        start_level=lambda df_: df_["start_level"].astype(float),
        current_level=lambda df_: df_["current_level"].astype(float),
        date_of_birth=lambda df_: pd.to_datetime(df_["date_of_birth"]),
        start_date=lambda df_: pd.to_datetime(df_["start_date"]),
        end_date=lambda df_: pd.to_datetime(df_["end_date"]),
        email=lambda df_: df_["email"].str.lower().str.strip(),
        mobile=lambda df_: module.clean_phone_number(df_["mobile"]),
        consultant = lambda df_: df_["consultant"].str.upper(),
        is_cpt = lambda df_: module.get_cpt(df_), 
        student_center = lambda df_: module.get_center(df_),
        student_area = lambda df_: module.get_area(df_),
    )
    # ! drop ST
    .loc[
        lambda df_: ~(
            df_["student_code"].str.contains("STREET TALK|STREETTALK", na=False)
        )
    ]
    # ! drop duplicated member based on student code and start date
    .drop_duplicates(subset=["student_code", "start_date"], keep="first")
    # ! drop unnecessary cols
    .drop(
        columns=[
            "gender",
            "home",
            "work",
            "end_level",
            "on_track",
            "course_status",
            "personal_tutor",
            "first_name",
            "last_name",
            "center_name",
        ]
    )
)

/home/anj/Documents/wse-local/2. Experience/data - coco member/module.py:166: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (


# Assertion

In [82]:
assert sorted(df_clean["student_membership"].unique()) == ["Deluxe", "GO", "VIP"]

# Save DF

In [83]:
filename = ("coco member.xlsx").replace(" ", "_")
output_path = Path("output", folder_path.stem)

if not os.path.exists(full_filepath := output_path / filename):
    try:
        os.mkdir(output_path)
    except FileExistsError:
        pass
    df_clean.to_excel(full_filepath, engine="xlsxwriter", index=False)
    print("File saved.")
else:
    print("File already exist.")

File saved.


# Experiment

In [84]:
# jkt_1 = ["PP", "SDC", "KG"]
# jkt_2 = ["GC", "LW", "BSD", "TBS"]
# jkt_3 = ["KK", "CBB", "SMB"]
# bdg = ["DG"]
# sby = ["PKW"]
# centers = jkt_1 + jkt_2 + jkt_3 + bdg + sby
# pattern = f'({"|".join(centers)})'

# (df_ori
#     .loc[~(df_ori["Last Name"].str.contains(pattern, regex=True, na=False))]
# )

In [85]:
# (df_clean
#     .loc[
#         (df_clean["student_center"].isna()) | (df_clean["student_area"].isna()) | (df_clean["student_membership"].isna()) |
#         (df_clean["student_center"] == "Not Specified") | (df_clean["student_area"] == "Not Specified") | (df_clean["student_membership"] == "Not Specified")
#     ]
#     .sample(10)
# )